In [29]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# Define movie_chunks with some example data
# Replace this with your actual list of Document objects
from langchain.schema import Document

tv_chunks = [
Document(page_content="Alone is a survival show.", metadata={"title": "Alone"})
]

# Define the embedding model
embedding_model = HuggingFaceEmbeddings()

# Create a FAISS vectorstore from your movie_chunks and embedding_model
vectorstore = FAISS.from_documents(tv_chunks, embedding_model)


C:\Users\jason\AppData\Local\Temp\ipykernel_38924\3668809234.py:13: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding_model = HuggingFaceEmbeddings()


In [30]:
from langchain.chains import ConversationalRetrievalChain
user_input = "I really like the TV show alone. Can you recommend other TV shows like that? I like survival shows but not game shows"
user_embedding = embedding_model.embed_query(user_input)


In [31]:
docs = vectorstore.similarity_search_by_vector(user_embedding, k=5)


In [32]:
import os

# Ensure openai_api_key is set, otherwise get from environment or prompt the user
if not openai_api_key:
	openai_api_key = os.getenv("OPENAI_API_KEY")
	if not openai_api_key:
		from getpass import getpass
		openai_api_key = getpass("Enter your OpenAI API key: ")

# Re-initialize chat_model with the API key
chat_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7, openai_api_key=openai_api_key)

# Re-initialize the chain with the new chat_model if needed
chain = LLMChain(llm=chat_model, prompt=prompt)

# Run the recommendation
recommendation = chain.run(input=user_input, results=docs)
print(recommendation)


Based on your preference for survival shows like "Alone," I recommend the following TV shows and movies:

1. "Man vs. Wild" - This show follows survival expert Bear Grylls as he demonstrates various techniques for surviving in extreme environments. It provides a mix of educational content and thrilling survival challenges.

2. "Survivorman" - This series features survival expert Les Stroud as he documents his experiences surviving alone in various remote locations. It offers a realistic portrayal of the challenges and skills needed to survive in the wilderness.

3. "Into the Wild" (movie) - This film tells the true story of Christopher McCandless, a young man who gives up his possessions and embarks on a journey into the Alaskan wilderness. It explores themes of self-discovery, adventure, and survival in a harsh environment.

These recommendations align with your interest in survival shows that focus on real-life challenges and skills, rather than competitive game shows. They offer eng

In [33]:
import gradio as gr

def recommend_tv_shows(user_input):
    # Embed the user input
    user_embedding = embedding_model.embed_query(user_input)
    # Search for similar documents
    docs = vectorstore.similarity_search_by_vector(user_embedding, k=5)
    # Run the LLM chain for recommendations
    recommendation = chain.run(input=user_input, results=docs)
    return recommendation

iface = gr.Interface(
    fn=recommend_tv_shows,
    inputs=gr.Textbox(lines=3, label="Describe your favorite TV shows or preferences"),
    outputs=gr.Textbox(label="Recommendations"),
    title="TV & Movie Recommendation Chatbot",
    description="Get personalized TV show and movie recommendations based on your preferences."
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [20]:
# Update the prompt template to avoid referencing specific titles like "Inception" or "Alone"
prompt = PromptTemplate(
    input_variables=["input", "results"],
    template="Based only on the user's taste profile: {input}, recommend TV shows or movies from {results} and justify each. Do not reference any specific titles from the user's input."
)

# Re-initialize the chain with the updated prompt
chain = LLMChain(llm=chat_model, prompt=prompt)